<a href="https://colab.research.google.com/github/SiddheshJadhav31/OffTargetPrediction_CrisprCas9/blob/main/Crispr_Cas9_CNN_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
circle_seq = pd.read_csv('/content/drive/MyDrive/Crispr_DatasetKaggle/CIRCLE_seq.csv')
guide_seq = pd.read_csv('/content/drive/MyDrive/Crispr_DatasetKaggle/GUIDE-Seq.csv')
mismatch_data = pd.read_csv('/content/drive/MyDrive/Crispr_DatasetKaggle/Haeussler.csv')

In [5]:
print("CircleSEQ" ,circle_seq.info())
print("GuideSEQ" ,guide_seq.info())
print("MismatchedHaussler" , mismatch_data.info())
fraction = 0.2


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584949 entries, 0 to 584948
Data columns (total 7 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   sgRNA_seq        584949 non-null  object 
 1   off_seq          584949 non-null  object 
 2   label            584949 non-null  float64
 3   Read             584949 non-null  float64
 4   sgRNA_type       584949 non-null  object 
 5   Cell             584949 non-null  object 
 6   Read_normalised  584949 non-null  float64
dtypes: float64(3), object(4)
memory usage: 31.2+ MB
CircleSEQ None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 213933 entries, 0 to 213932
Data columns (total 5 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   DNA     213933 non-null  object 
 1   crRNA   213933 non-null  object 
 2   label   213933 non-null  float64
 3   read    213933 non-null  float64
 4   pair    213933 non-null  object 
dtypes: float64(2), obj

In [6]:
print(circle_seq.duplicated().sum())
print(guide_seq.duplicated().sum())
print(mismatch_data.duplicated().sum())
circle_seq = circle_seq.drop_duplicates()
guide_seq = guide_seq.drop_duplicates()
mismatch_data = mismatch_data.drop_duplicates()


317
40798
0


In [7]:
def get_complementary_sequence(sequence):
    # Create a translation table: A <-> T, C <-> G
    complement = str.maketrans('ATCG', 'TAGC')

    # Translate the sequence and reverse it to get the target (complementary sequence)
    return sequence.translate(complement)[::-1]

In [8]:
reduced_circle_seq = circle_seq.sample(frac=fraction, random_state=42)

In [9]:
reduced_circle_seq['target'] = reduced_circle_seq['sgRNA_seq'].apply(get_complementary_sequence)

In [10]:
def one_hot_encode_sequence(sequence):
    # Define the encoding map for standard nucleotides
    encoding_map = {
        'A': [1, 0, 0, 0],
        'T': [0, 1, 0, 0],
        'C': [0, 0, 1, 0],
        'G': [0, 0, 0, 1],
        '_': [0, 0, 0, 0]  # Use [0, 0, 0, 0] for underscores or other non-standard characters
    }

    # Default to [0, 0, 0, 0] for any characters not in the map
    one_hot_encoded = [encoding_map.get(nuc, [0, 0, 0, 0]) for nuc in sequence]

    # Flatten the list of lists into a single list
    return [item for sublist in one_hot_encoded for item in sublist]

In [11]:
def one_hot_encode_dataframe(df, sequence_columns):
    # Iterate through each sequence column for encoding
    for col in sequence_columns:
        # Create a DataFrame of the one-hot encoded sequences for each column
        one_hot_encoded_df = df[col].apply(lambda seq: one_hot_encode_sequence(seq))

        # Convert the list of encoded sequences into a DataFrame
        one_hot_encoded_df = pd.DataFrame(
            one_hot_encoded_df.tolist(),
            index=df.index
        )

        # Rename columns to include the original column name as a prefix for clarity
        one_hot_encoded_df.columns = [
            f"{col}_pos_{i+1}" for i in range(one_hot_encoded_df.shape[1])
        ]

        # Add the encoded columns to the original DataFrame
        df = pd.concat([df, one_hot_encoded_df], axis=1)

        # Drop the original sequence column if you no longer need it
        df.drop(columns=[col], inplace=True)

    return df


In [12]:
# Define the columns in circle_seq that need to be one-hot encoded
sequence_columns = ['sgRNA_seq', 'off_seq','sgRNA_type', 'target']  # Include any other sequence columns if present

# Apply the one-hot encoding function to these columns
encoded_circle_seq = one_hot_encode_dataframe(reduced_circle_seq, sequence_columns)

In [13]:
# Define the columns containing nucleotide sequences and pairs in guide_seq
sequence_columns_guide_seq = ['DNA', 'crRNA', 'pair']

# Apply the one-hot encoding function to these columns
encoded_guide_seq = one_hot_encode_dataframe(guide_seq, sequence_columns_guide_seq)

In [14]:
# Define the input features
input_columns = [col for col in encoded_circle_seq.columns if 'sgRNA_seq_pos_' in col or 'off_seq_pos_' in col or 'sgRNA_type_pos_' in col]
X = encoded_circle_seq[input_columns]

In [15]:
print(X.columns.tolist())

['sgRNA_seq_pos_1', 'sgRNA_seq_pos_2', 'sgRNA_seq_pos_3', 'sgRNA_seq_pos_4', 'sgRNA_seq_pos_5', 'sgRNA_seq_pos_6', 'sgRNA_seq_pos_7', 'sgRNA_seq_pos_8', 'sgRNA_seq_pos_9', 'sgRNA_seq_pos_10', 'sgRNA_seq_pos_11', 'sgRNA_seq_pos_12', 'sgRNA_seq_pos_13', 'sgRNA_seq_pos_14', 'sgRNA_seq_pos_15', 'sgRNA_seq_pos_16', 'sgRNA_seq_pos_17', 'sgRNA_seq_pos_18', 'sgRNA_seq_pos_19', 'sgRNA_seq_pos_20', 'sgRNA_seq_pos_21', 'sgRNA_seq_pos_22', 'sgRNA_seq_pos_23', 'sgRNA_seq_pos_24', 'sgRNA_seq_pos_25', 'sgRNA_seq_pos_26', 'sgRNA_seq_pos_27', 'sgRNA_seq_pos_28', 'sgRNA_seq_pos_29', 'sgRNA_seq_pos_30', 'sgRNA_seq_pos_31', 'sgRNA_seq_pos_32', 'sgRNA_seq_pos_33', 'sgRNA_seq_pos_34', 'sgRNA_seq_pos_35', 'sgRNA_seq_pos_36', 'sgRNA_seq_pos_37', 'sgRNA_seq_pos_38', 'sgRNA_seq_pos_39', 'sgRNA_seq_pos_40', 'sgRNA_seq_pos_41', 'sgRNA_seq_pos_42', 'sgRNA_seq_pos_43', 'sgRNA_seq_pos_44', 'sgRNA_seq_pos_45', 'sgRNA_seq_pos_46', 'sgRNA_seq_pos_47', 'sgRNA_seq_pos_48', 'sgRNA_seq_pos_49', 'sgRNA_seq_pos_50', 'sgRNA_s

In [16]:
y = encoded_circle_seq.filter(like='target_pos_')

In [17]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
X_train = X_train.values.reshape(X_train.shape[0], -1, 4)  # Change -1 to the sequence length if known
X_test = X_test.values.reshape(X_test.shape[0], -1, 4)

print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(93540, 71, 4) (23386, 71, 4) (93540, 96) (23386, 96)


In [23]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from keras.regularizers import l2

# Define a simplified model
model = Sequential()

# First convolutional layer
model.add(Conv1D(32, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))  # Higher dropout rate

# Second convolutional layer
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))  # Higher dropout rate

# Third convolutional layer (optional, before flattening)
model.add(Conv1D(256, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.5))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu', kernel_regularizer=l2(0.001)))
model.add(Dropout(0.5))  # Higher dropout rate
model.add(Dense(y_train.shape[1], activation='sigmoid'))  # Sigmoid for multi-label classification

# Compile model
optimizer = Adam(learning_rate=0.00001)  # Keep a lower learning rate
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)




In [24]:
# Fit the model
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping]
)

Epoch 1/50
2924/2924 [==============================] - 29s 10ms/step - loss: 0.7097 - accuracy: 0.0229 - val_loss: 0.5791 - val_accuracy: 0.0000e+00
Epoch 2/50
2924/2924 [==============================] - 28s 9ms/step - loss: 0.4638 - accuracy: 0.0469 - val_loss: 0.3974 - val_accuracy: 0.0000e+00
Epoch 3/50
2924/2924 [==============================] - 27s 9ms/step - loss: 0.3523 - accuracy: 0.0773 - val_loss: 0.2891 - val_accuracy: 0.0000e+00
Epoch 4/50
2924/2924 [==============================] - 28s 10ms/step - loss: 0.2787 - accuracy: 0.1158 - val_loss: 0.2239 - val_accuracy: 0.0000e+00
Epoch 5/50
2924/2924 [==============================] - 28s 10ms/step - loss: 0.2345 - accuracy: 0.1401 - val_loss: 0.1838 - val_accuracy: 0.0000e+00
Epoch 6/50
2924/2924 [==============================] - 28s 10ms/step - loss: 0.2022 - accuracy: 0.1661 - val_loss: 0.1548 - val_accuracy: 0.0000e+00
Epoch 7/50
2924/2924 [==============================] - 28s 10ms/step - loss: 0.1786 - accuracy: 0.192

In [25]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)

print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

731/731 [==============================] - 2s 3ms/step - loss: 0.0441 - accuracy: 0.5134
Test Loss: 0.0441
Test Accuracy: 0.5134


In [27]:
from sklearn.metrics import classification_report

# Get predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)  # Use a threshold of 0.5 for multi-label

# Define your class names based on your dataset
num_classes = y_test.shape[1]  # Assuming y_test has the shape (num_samples, num_classes)
target_names = [f'target_pos_{i}' for i in range(num_classes)]

# Generate the classification report
print(classification_report(y_test, y_pred, target_names=target_names))


731/731 [==============================] - 2s 3ms/step
               precision    recall  f1-score   support

 target_pos_0       0.00      0.00      0.00         0
 target_pos_1       0.00      0.00      0.00         0
 target_pos_2       1.00      1.00      1.00     23386
 target_pos_3       0.00      0.00      0.00         0
 target_pos_4       0.00      0.00      0.00         0
 target_pos_5       0.00      0.00      0.00         0
 target_pos_6       1.00      1.00      1.00     23386
 target_pos_7       0.00      0.00      0.00         0
 target_pos_8       0.97      0.91      0.94      7572
 target_pos_9       0.95      0.75      0.84      6190
target_pos_10       0.97      0.38      0.54      5264
target_pos_11       0.96      0.43      0.59      4360
target_pos_12       1.00      1.00      1.00       927
target_pos_13       1.00      1.00      1.00      3982
target_pos_14       1.00      1.00      1.00      9882
target_pos_15       1.00      1.00      1.00      8595
target_po

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [28]:
reduced_circle_seq.head(50)

,sgRNA_seq,off_seq,label,Read,sgRNA_type,Cell,Read_normalised,target
63864,GGAATCCCTTCTGC_AGCACCCGG,GGAATCCCTTCTCCAAAGACCCAC,0.0,0.0,GGAATCCCTTCTGCAGCACCNGG,,0.0,CCGGGTGCT_GCAGAAGGGATTCC
162369,GG_TGAGTGAGTGTGTGCGTGGGG,GGGTGGGTGTGTGTGTGTGTGGCT,0.0,0.0,GGTGAGTGAGTGTGTGCGTGNGG,,0.0,CCCCACGCACACACTCACTCA_CC
78678,-GGGTGGGGGGAGTTTGCTCCAGG,-GGGTATGGGGAGTTAGCT_CATA,0.0,0.0,GGGTGGGGGGAGTTTGCTCCNGG,,0.0,CCTGGAGCAAACTCCCCCCACCC-
355168,-GGAATCCCTTCTGCAGCACCTGG,-AGAAGCCCTTCTGCGTTACCTGG,0.0,0.0,GGAATCCCTTCTGCAGCACCNGG,,0.0,CCAGGTGCTGCAGAAGGGATTCC-
462524,-GGTGAGTGAGTGTGTGCGTGTGG,-TGGTTGTGTGTGTGTGCGTGTGT,0.0,0.0,GGTGAGTGAGTGTGTGCGTGNGG,,0.0,CCACACGCACACACTCACTCACC-
474214,-GGCCCAGACTGAGCACGTGATGG,-GGCCCAGTCAGAACACCTCATGC,0.0,0.0,GGCCCAGACTGAGCACGTGANGG,,0.0,CCATCACGTGCTCAGTCTGGGCC-
136780,-GAACACAAAGCATAGACTGCTGG,-GAACA_AAAGCTCAGACCGCTGA,0.0,0.0,GAACACAAAGCATAGACTGCNGG,,0.0,CCAGCAGTCTATGCTTTGTGTTC-
27174,GG_TGAGTGAGTGTGTGCGTGTGG,GGGTGTGTGTGTGTGTGCGTGTAT,0.0,0.0,GGTGAGTGAGTGTGTGCGTGNGG,,0.0,CCACACGCACACACTCACTCA_CC
317550,-GGGTGGGGGGAGTTTGCTCCTGG,-GGGTCGGGGGCGGTTGTCCTTGG,0.0,0.0,GGGTGGGGGGAGTTTGCTCCNGG,,0.0,CCAGGAGCAAACTCCCCCCACCC-
580824,-GACCCCCTCCACCCCGCCTCAGG,-CGCTCCCCCCACCCCACCACAGG,1.0,96.0,GACCCCCTCCACCCCGCCTCNGG,U2OS,96.0,CCTGAGGCGGGGTGGAGGGGGTC-
